## 2023 Open Science Data Challenge - Sentinel-1 Phenology

This notebook calculates vegetation phenology changes using radiometrically terrain corrected Sentinel-1 radar data. To detect vegetation changes, the algorithm uses variations in the Radar Vegetation Index (RVI) which is a common proxy for vegetation growth and health. The outputs of this notebook can be used to assess differences in agriculture fields over time or space and also allow the assessment of growing states such as planting and harvesting. The baseline data is [Sentinel-1 RTC](https://planetarycomputer.microsoft.com/dataset/sentinel-1-rtc) data from the MS Planetary Computer catalog. For more information about how to interpret radar data, see the following document: "A Layman's Interpretation Guide to L-Band and C-Band Synthetic Aperture Radar Data" found <a href="https://ceos.org/document_management/SEO/DataCube/Laymans_SAR_Interpretation_Guide_2.0.pdf" target="_blank"><b>HERE</b></a>.


In [1]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio.features

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer as pc
# import xrspatial.multispectral as ms
import odc
from odc.stac import stac_load

# Pass your API key here
pc.settings.set_subscription_key('cf5657d28bb2408ba8fd775642c2e1cb')

### Discover and load the data for analysis

Our test region is a rice crop field in Vietnam. First, we define our area of interest using latitude and longitude coordinates of the centroid. Then we define the size of the surrounding bounding box (in degrees). GeoJSON format uses a specific order: (longitude, latitude), so be careful when entering the coordinates.

In [2]:
# Rice Crop Field in An Giang, Vietnam

# lat_long = (10.4391, 105.3338) # Lat-Lon centroid location
lat_long_rice = (10.322364360592521, 105.27843410554115)		
lat_long_non_rice = (10.013942985253381, 105.67361318732796)	
box_size_deg_3x3 = 0.0002 # Surrounding box in degrees, yields approximately 3x3 pixel region
box_size_deg_5x5 = 0.0004 # Surrounding box in degrees, yields approximately 5x5 pixel region
box_size_deg_7x7 = 0.0006 # Surrounding box in degrees, yields approximately 7x7 pixel region

def create_bbox(lat_long, box_size_deg):
    min_lon = lat_long[1]-box_size_deg/2
    min_lat = lat_long[0]-box_size_deg/2
    max_lon = lat_long[1]+box_size_deg/2
    max_lat = lat_long[0]+box_size_deg/2
    return (min_lon, min_lat, max_lon, max_lat)


bbox_3x3_rice = create_bbox(lat_long_rice, box_size_deg_3x3)
bbox_3x3_non_rice = create_bbox(lat_long_non_rice, box_size_deg_3x3)
# bbox_5x5 = create_bbox(lat_long, box_size_deg_5x5)
# bbox_7x7 = create_bbox(lat_long, box_size_deg_7x7)

Using the `pystac_client` we can search the Planetary Computer's STAC endpoint for items matching our query parameters. We will use a time window in late-2021 to early-2022 when we know there is a rice crop growing cycle. The result is the number of scenes matching our search criteria that touch our area of interest. Some of these may be partial scenes. 

In [3]:
time_of_interest = "2021-11-01/2022-11-11"

In [4]:
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
search_rice = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox_3x3_rice, datetime=time_of_interest)
search_non_rice = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox_3x3_non_rice, datetime=time_of_interest)
items_rice = list(search_rice.get_all_items()) # This produces a list of scene IDs
items_non_rice = list(search_non_rice.get_all_items()) # This produces a list of scene IDs

Next, we'll load the data into an [xarray](https://xarray.pydata.org/en/stable/) using [stackstac](https://stackstac.readthedocs.io/) and then "clip" the data to only the pixels within our region (bounding box). We will only keep the desired bands (VV, VH) since they are needed to create the Radar Vegetation Index (RVI). There are also several other <b>important settings for the data</b>: We have changed the projection to epsg=4326 which is standard latitude-longitude in degrees and we have specified the spatial resolution of each pixel to be 10-meters, which is the baseline accuracy for this data. 

In [5]:
# Define the pixel resolution for the final product
# Define the scale according to our selected crs, so we will use degrees

resolution = 10  # meters per pixel 
scale = resolution / 111320.0 # degrees per pixel for crs=4326 

In [6]:
# Load the data using Open Data Cube
data_rice = stac_load(items_rice,bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox_3x3_rice)
data_non_rice = stac_load(items_non_rice,bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox_3x3_non_rice)

### Radar Vegetation Index (RVI)

The <b>Radar Vegetation Index (RVI)</b> is one example of an index used to measure vegetation growth with radar data. Since radar data can penetrate clouds, such indices are valuable to monitor crop phenology (growth). For more information on the RVI formula, see the following Sentinel Hub link: <a href="https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-1/radar_vegetation_index/" target="_blank"><b>HERE</b></a>.


In [7]:
# Calculate the mean of the data across the sample region
mean_rice = data_rice.mean(dim=['y', 'x']).compute()
mean_non_rice = data_non_rice.mean(dim=['y', 'x']).compute()

In [11]:
# Calculate RVI
def rvi(mean):
    dop = (mean.vv / (mean.vv + mean.vh))
    m = 1 - dop
    rvi = (np.sqrt(dop))*((4*mean.vh)/(mean.vv + mean.vh))
    return rvi

rvi_rice = rvi(mean_rice)
rvi_non_rice = rvi(mean_non_rice)

In [12]:
fig = plt.figure(figsize=(10, 5))
rvi_non_rice.plot(color='green',marker='o',markersize=4,linewidth=2)
plt.title("Mean RVI = Radar Vegetation Index Non Rice")
plt.xlabel("Date")
plt.ylabel("RVI")
plt.show()

In [13]:
fig = plt.figure(figsize=(10, 5))
rvi_rice.plot(color='green',marker='o',markersize=4,linewidth=2)
plt.title("Mean RVI = Radar Vegetation Index Rice")
plt.xlabel("Date")
plt.ylabel("RVI")
plt.show()